In [1]:
import pandas as pd
import numpy as np
import os, random, shutil, re
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from copy import deepcopy

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
def check_NA(df):
    print(f"NA check")
    NA_res = df.isna().sum()
    if NA_res.sum() != 0:
        for idx, i in enumerate(NA_res):
            if i != 0:
                print(df.columns[idx],i,"->",round(i/df.shape[0]*100,3),"%")
    else:
        print("...There's no NA")

def get_ls(filepath):
    return [f for f in listdir(filepath) if isfile(join(filepath, f))]

def get_res(filepath,filename):
    df = pd.read_csv(os.path.join(filepath,filename))
    if len(df.columns)==3:
        df.columns = ['day','hour','value']
    else:
        df.columns = ['day','hour','e','value']
        df = df[['day','hour','value']]
    df['emd_nm'] = re.findall('.*동|.*면|.*읍', filename)[0]
    df['type'] = filename.split("_")[-3]
    df['day'] = df['day'].apply(lambda x: x.strip()[8:] if "Start" in x  else x.strip())
    df['month'] = 0
    ls = list(filter(lambda x: len(x)>3, df['day'].unique()))
    s = 0
    e = df.loc[df.day==ls[0]].index[0]
    df['month'][s:e] = 1

    for idx in range(len(ls)):    
        s = df.loc[df.day==ls[idx]].index[0]+1
        if idx == len(ls)-1:
            e = df.shape[0]
        else:
            e = df.loc[df.day==ls[idx+1]].index[0]
        df['month'][s:e] = pd.to_datetime(ls[idx]).month
    df['day'] = df['day'].astype('int')
    df['year'] = pd.to_datetime(ls[idx]).year
    df = df[['year','month', 'day','hour','emd_nm','type','value']]
    df = df.dropna()
    return df

def get_res_for_8(filepath,filename):
    df = pd.read_csv(os.path.join(filepath,filename))
    if len(df.columns)==3:
        df.columns = ['day','hour','value']
    else:
        df.columns = ['day','hour','e','value']
        df = df[['day','hour','value']]
    df['emd_nm'] = re.findall('.*동|.*면|.*읍', filename)[0]
    df['type'] = filename.split("_")[-3]
    df['day'] = df['day'].apply(lambda x: x.strip()[8:] if "Start" in x  else x.strip())
    df['month'] = 8
    df['day'] = df['day'].astype('int')
    df['year'] = 2021
    df = df[['year','month', 'day','hour','emd_nm','type','value']]
    df = df.dropna()
    return df

def get_df(filepath,files, df):
    for filename in tqdm(files):
        res = get_res(filepath,filename)
        df = pd.concat([df, res], axis=0)
    print(df.shape)
    return df

def get_df_for_8(filepath,files, df):
    for filename in tqdm(files):
        res = get_res_for_8(filepath,filename)
        df = pd.concat([df, res], axis=0)
    print(df.shape)
    return df

In [3]:
for y in ["18","19","20","21","2108"]:
    filepath = './data/기상'+y+"/"
    files = "files_"+y
    exec("%s = get_ls('%s')" %(files, filepath))

In [4]:
y ="18"
df = pd.DataFrame()
df = get_df('./data/기상'+y+"/",files_18, df)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:11<00:00, 13.93it/s]

(2016216, 7)


In [5]:
y ="19"
df = get_df('./data/기상'+y+"/",files_19, df)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:23<00:00,  7.04it/s]

(4291224, 7)


In [6]:
y ="20"
df = get_df('./data/기상'+y+"/",files_20, df)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:36<00:00,  4.49it/s]

(7138182, 7)


In [7]:
y ="21"
df = get_df('./data/기상'+y+"/",files_21, df)

100%|████████████████████████████████████████████████████████████████████████████████| 165/165 [00:46<00:00,  3.58it/s]

(8659614, 7)


In [10]:
y ="2108"
df = get_df_for_8('./data/기상'+y+"/",files_2108, df)

100%|████████████████████████████████████████████████████████████████████████████████| 123/123 [00:31<00:00,  3.90it/s]

(9208686, 7)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9208686 entries, 0 to 4493
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   year    int64  
 1   month   int64  
 2   day     int32  
 3   hour    float64
 4   emd_nm  object 
 5   type    object 
 6   value   float64
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 526.9+ MB


In [17]:
df = df.loc[df.type!="풍향",]

In [18]:
df.type.unique()

array(['강수', '기온', '습도', '풍속'], dtype=object)

In [19]:
check_NA(df)

NA check
...There's no NA


In [20]:
df = pd.read_csv(os.path.join(filepath, 'weather.csv'))
df.head()

,year,month,day,hour,emd_nm,type,value
0,2018,1,1,0.0,건입동,강수,0.0
1,2018,1,1,100.0,건입동,강수,0.0
2,2018,1,1,200.0,건입동,강수,0.0
3,2018,1,1,300.0,건입동,강수,0.0
4,2018,1,1,400.0,건입동,강수,0.0


In [21]:
df_weather = deepcopy(df)

# 이상치 대체
기온이 영하 50도인 경우와 강수, 습도, 풍속이 -1인 경우는 NA로 간주하고 linear interpolation으로 대체함

In [22]:
df_weather.loc[df_weather.value<-20,'value'] = np.NaN
df_weather.loc[(df_weather.type.isin(['강수','습도','풍속']))&(df_weather.value==-1),'value'] = np.NaN
df_weather['value'] = df_weather['value'].interpolate(method="linear")

In [23]:
df_weather.loc[df_weather.value>100,'type'].unique()

array([], dtype=object)

In [16]:
df_weather.groupby(['year','month','day','emd_nm','type']).mean()[['value']]

value
year month day emd_nm type           
2018 1     1   건입동    강수     0.000000
                      기온     5.254167
                      습도    54.041667
                      풍속     2.404167
               구좌읍    강수     0.000000
...                               ...
2021 8     31  화북동    기온    27.305556
                      습도    74.930556
               효돈동    강수     0.002778
                      기온    25.840278
                      습도    85.347222

[214758 rows x 1 columns]

In [17]:
df_weather['month'] = df_weather['month'].apply(lambda x: "{:0>2d}".format(x))
df_weather['day'] = df_weather['day'].apply(lambda x: "{:0>2d}".format(x))

In [18]:
df_weather['year'] = df_weather['year'].astype("str")
df_weather['month'] = df_weather['month'].astype("str")
df_weather['day'] = df_weather['day'].astype("str")

In [19]:
df_weather['base_date'] = df_weather[['year','month','day']].apply(lambda x: '-'.join(x), axis=1)

In [20]:
df_weather_ = df_weather[['base_date','emd_nm','type','value']]
df_weather_ 

,base_date,emd_nm,type,value
0,2018-01-01,건입동,강수,0.0
1,2018-01-01,건입동,강수,0.0
2,2018-01-01,건입동,강수,0.0
3,2018-01-01,건입동,강수,0.0
4,2018-01-01,건입동,강수,0.0
...,...,...,...,...
9203593,2021-08-31,효돈동,습도,85.0
9203594,2021-08-31,효돈동,습도,85.0
9203595,2021-08-31,효돈동,습도,85.0
9203596,2021-08-31,효돈동,습도,85.0


In [21]:
weather = df_weather_.pivot_table(index=['base_date','emd_nm'],columns="type",values="value")

In [23]:
weather.reset_index().to_csv(os.path.join(filepath, 'weather_preprocessed.csv'), index=False)

In [24]:
weather_ = pd.read_csv(os.path.join(filepath, 'weather_preprocessed.csv'))
weather_

,base_date,emd_nm,강수,기온,습도,풍속
0,2018-01-01,건입동,0.000000,5.254167,54.041667,2.404167
1,2018-01-01,구좌읍,0.000000,4.575000,63.333333,4.688889
2,2018-01-01,남원읍,0.000000,7.212500,51.375000,3.481944
3,2018-01-01,노형동,0.000000,5.379167,54.083333,3.181944
4,2018-01-01,대륜동,0.000000,8.287500,53.625000,2.404167
...,...,...,...,...,...,...
54853,2021-08-31,표선면,0.000000,27.243056,82.916667,NaN
54854,2021-08-31,한경면,0.000000,27.326389,92.361111,NaN
54855,2021-08-31,한림읍,0.001389,27.319444,91.076389,NaN
54856,2021-08-31,화북동,0.002778,27.305556,74.930556,NaN


# +) 누락된 풍속 데이터 추가 
2021.09.12.수정

In [79]:
weather_.loc[weather_.풍속.isna(),"base_date"].unique()

array(['2018-09-14', '2018-09-15', '2018-09-16', '2018-09-17',
       '2018-09-18', '2018-09-19', '2018-09-20', '2018-09-21',
       '2018-09-22', '2018-09-23', '2018-09-24', '2018-09-25',
       '2018-09-26', '2018-09-27', '2018-09-28', '2018-09-29',
       '2018-09-30', '2018-11-01', '2018-11-02', '2018-11-03',
       '2018-11-04', '2018-11-05', '2018-11-06', '2018-11-07',
       '2018-11-08', '2018-11-09', '2018-11-10', '2018-11-11',
       '2018-11-12', '2018-11-13', '2018-11-14', '2018-11-15',
       '2018-11-16', '2018-11-17', '2018-11-18', '2018-11-19',
       '2018-11-20', '2018-11-21', '2018-11-22', '2018-11-23',
       '2018-11-24', '2018-11-25', '2018-11-26', '2018-11-27',
       '2018-11-28', '2018-11-29', '2018-11-30', '2020-08-26',
       '2020-08-27', '2020-08-28', '2020-08-29', '2020-08-30',
       '2020-08-31', '2021-07-01', '2021-07-02', '2021-07-03',
       '2021-07-04', '2021-07-05', '2021-07-06', '2021-07-07',
       '2021-07-08', '2021-07-09', '2021-07-10', '2021-

In [24]:
def get_ls(filepath):
    return [f for f in listdir(filepath) if isfile(join(filepath, f))]

def get_res2(filepath,filename,year,month):
    df = pd.read_csv(os.path.join(filepath,filename))
    if len(df.columns)==3:
        df.columns = ['day','hour','value']
    else:
        df.columns = ['day','hour','e','value']
        df = df[['day','hour','value']]
    df['emd_nm'] = re.findall('.*동|.*면|.*읍', filename)[0]
    df['type'] = filename.split("_")[-3]
    df['month'] = month
    df['day'] = df['day'].astype('int')
    df['year'] = year
    df = df[['year','month', 'day','hour','emd_nm','type','value']]
    df = df.dropna()
    return df

def get_df2(filepath,files, df,year,month):
    for filename in tqdm(files):
        res = get_res2(filepath,filename,year,month)
        df = pd.concat([df, res], axis=0)
    print(df.shape)
    return df

In [70]:
for y in ["풍속2008추가","풍속1811","풍속201809","풍속21"]:
    filepath = './data/기상'+y+"/"
    files = "files_"+y
    exec("%s = get_ls('%s')" %(files, filepath))

In [43]:
y ="풍속2008추가"
df = pd.DataFrame()
df = get_df2('./data/기상'+y+"/",files_풍속2008추가, df, 2020, 8)

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 61.98it/s]

(197284, 7)


In [44]:
y ="풍속1811"
df = get_df2('./data/기상'+y+"/",files_풍속1811, df, 2018, 11)

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 47.04it/s]

(388204, 7)


In [45]:
y ="풍속201809"
df = get_df2('./data/기상'+y+"/",files_풍속201809, df, 2018, 9)

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:01<00:00, 34.74it/s]

(579124, 7)


In [30]:
df_ori = pd.read_csv('./data/weather.csv')
df_ori.head()

,year,month,day,hour,emd_nm,type,value
0,2018,1,1,0.0,건입동,강수,0.0
1,2018,1,1,100.0,건입동,강수,0.0
2,2018,1,1,200.0,건입동,강수,0.0
3,2018,1,1,300.0,건입동,강수,0.0
4,2018,1,1,400.0,건입동,강수,0.0


In [73]:
df_ori.loc[(df_ori.year==2021)&(df_ori.month==8)&(df_ori.type=="기온")]

,year,month,day,hour,emd_nm,type,value
8658990,2021,8,1,30.0,건입동,기온,28.0
8658991,2021,8,1,30.0,건입동,기온,29.0
8658992,2021,8,1,30.0,건입동,기온,29.0
8658993,2021,8,1,30.0,건입동,기온,31.0
8658994,2021,8,1,30.0,건입동,기온,31.0
...,...,...,...,...,...,...,...
9199129,2021,8,31,2330.0,효돈동,기온,26.0
9199130,2021,8,31,2330.0,효돈동,기온,26.0
9199131,2021,8,31,2330.0,효돈동,기온,27.0
9199132,2021,8,31,2330.0,효돈동,기온,28.0


In [32]:
df = pd.concat([df_ori, df], axis=0)

In [33]:
df_weather = deepcopy(df)

In [34]:
df_weather['month'] = df_weather['month'].apply(lambda x: "{:0>2d}".format(x))
df_weather['day'] = df_weather['day'].apply(lambda x: "{:0>2d}".format(x))

In [35]:
df_weather['year'] = df_weather['year'].astype("str")
df_weather['month'] = df_weather['month'].astype("str")
df_weather['day'] = df_weather['day'].astype("str")

In [36]:
df_weather['base_date'] = df_weather[['year','month','day']].apply(lambda x: '-'.join(x), axis=1)

In [37]:
df_weather_ = df_weather[['base_date','emd_nm','type','value']]
df_weather_ 

,base_date,emd_nm,type,value
0,2018-01-01,건입동,강수,0.0
1,2018-01-01,건입동,강수,0.0
2,2018-01-01,건입동,강수,0.0
3,2018-01-01,건입동,강수,0.0
4,2018-01-01,건입동,강수,0.0
...,...,...,...,...
8984,2021-08-31,효돈동,풍속,5.0
8985,2021-08-31,효돈동,풍속,5.0
8986,2021-08-31,효돈동,풍속,6.0
8987,2021-08-31,효돈동,풍속,6.0


In [86]:
weather = df_weather_.pivot_table(index=['base_date','emd_nm'],columns="type",values="value").reset_index()

In [87]:
weather = weather.loc[~weather.emd_nm.isin(["우도면","추자면"]),]

In [88]:
check_NA(weather)

NA check
풍속 41 -> 0.075 %


In [89]:
na_ls = []
for files in files_풍속21:
    tmp = pd.read_csv('./data/기상풍속21'+"/"+files)
    na_ls.append(np.mean(tmp.iloc[:144,3:4]).values[0])
weather.loc[weather.풍속.isna(),'풍속'] = na_ls

In [90]:
check_NA(weather)

NA check
...There's no NA


In [91]:
weather.to_csv(os.path.join(filepath, 'weather_preprocessed.csv'),index=False)
weather_final = pd.read_csv(os.path.join(filepath, 'weather_preprocessed.csv'))

In [92]:
weather_final

,base_date,emd_nm,강수,기온,습도,풍속
0,2018-01-01,건입동,0.000000,5.254167,54.041667,2.404167
1,2018-01-01,구좌읍,0.000000,4.575000,63.333333,4.688889
2,2018-01-01,남원읍,0.000000,7.212500,51.375000,3.481944
3,2018-01-01,노형동,0.000000,5.379167,54.083333,3.181944
4,2018-01-01,대륜동,0.000000,8.287500,53.625000,2.404167
...,...,...,...,...,...,...
54894,2021-08-31,표선면,0.000000,27.243056,82.916667,4.555556
54895,2021-08-31,한경면,0.000000,27.326389,92.361111,5.131944
54896,2021-08-31,한림읍,0.001389,27.319444,91.076389,3.937500
54897,2021-08-31,화북동,0.002778,27.305556,74.930556,2.430556
